In [7]:
import import_ipynb
import cnn_genre_classifier_spectrograms as mgr
import torch
import numpy as np
from pytorch_model_summary import summary


# path to file that stores spectrograms and genre labels for each processed segment
DATA_PATH = "/data/shared/GTZAN-DATASET/spectrograms_10segments.npy"

In [2]:
# load and split processed dataset

X_train, X_validation, X_test, y_train, y_validation, y_test = mgr.load_data(DATA_PATH, test_size=0.10, validation_size=0.10)

Loading data...
Data succesfully loaded!


In [10]:
# declare execution device

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")

else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [8]:
# load existing or create new model

model = mgr.new_model(device)
# model = mgr.load_model("./saved_models/model-1616675807trained.pth", device)

print(summary(model, torch.rand(1, 1, np.shape(X_train)[2], np.shape(X_train)[3]).to(device)), sep='\n')

Model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1))
  (norm3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1))
  (norm4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=8192, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (loss_function): CrossEntropyLoss()
)
-------------------------------------------------------------------------
      Layer (type)          Output Shape         Param #     Tr. Param #
          Conv2d-1     [1, 32, 5

In [11]:
# training pipeline

history = model.fit(
    X_train, y_train,
    validation_data=(X_validation, y_validation), 
    epochs=40, 
    batch_size=128,
    log=True)

mgr.plot_history(history)

print("Max_val_acc:", max(history['val_acc']), "  Max_train_acc:", max(history['acc']))
print("Min_val_loss:", min(history['val_loss']), "  Min_train_loss:", min(history['loss']))
    
model.test(X_test, y_test, out=True)

  0%|          | 0/40 [00:00<?, ?it/s]


NameError: name 'device' is not defined

In [ ]:
save_model(model, "./saved_models")